<a href="https://www.kaggle.com/code/gianpieroandrenacci/dostoevsky-s-crime-and-punishment-characters?scriptVersionId=154888545" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 📕 Identify Dostoevsky's Crime and Punishment characters

<blockquote style="color:#33001a; font-family:cursive;font-size:130%">I would like to live in a world where science and humanitas are just one thing</blockquote>

![Crime_And_Punishment.png](https://www.npensieri.it/wp-content/uploads/Delitto_E_Castigo.png)

📑  
* [1. Introduction](#1)
    * [1.1 Import Libraries](#1.1)
    * [1.2 Custom Functions](#1.2)
    * [1.3 Spacy Library](#1.3)
    * [1.4 Import the book](#1.4)
* [2. Characters Identification](#2)
    * [2.1 Clean the text](#2.1)
    * [2.2 Entitity Recognition](#2.2)
    * [2.3 Characters Entitity Recognition](#2.3)
    * [2.4 Improve Characters Identification](#2.4)
    * [2.5 Name Variation Resolution](#2.5)
    * [2.6 Final Word Cloud](#2.6)
    * [2.7 Reference](#2.7!)

<a id="1"></a>
<h1 style="background-color:#33001a;font-family:cursive;font-size:200%;color:#fff;text-align:center;border-radius:40px;height:40px;line-height:40px;">Introduction</h1>

<div class="alert alert-block alert alert-danger" style="font-size:14px; font-family:verdana; line-height: 1.7em; ">
    📌 &nbsp; Please keep in mind that, we are always on our learning journey. I just want to share something that will be interesting for you. If you find this notebook useful in anyway, please upvote it so that it can reach a bigger audience. You can share it with your fellow kagglers.
</div>

<p style="font-size:130%; background-color:#f28482;border-radius: 0px 30px;text-align:center;font-family:Sans-serif;font-weight:bold">"Crime and Punishment" by Fyodor Dostoevsky is a classic novel that features a wide range of characters. Here are the top 20 characters in the book:</p>

1. Rodion Romanovich Raskolnikov - The main protagonist of the novel who murders an old pawnbroker and her sister.
2. Avdotya Romanovna Raskolnikova (Dounia) - Raskolnikov's sister who is engaged to Luzhin.
3. Arkady Ivanovich Svidrigailov - A wealthy, powerful and mysterious man who is interested in Raskolnikov's sister Dounia.
4. Porfiry Petrovich - A shrewd police detective who is investigating the murders.
5. Sofya Semyonovna Marmeladova (Sonya) - A young prostitute who is forced into the trade to support her family.
6. Dmitri Prokofych Razumikhin - Raskolnikov's loyal and compassionate friend.
7. Pulcheria Alexandrovna Raskolnikova - Raskolnikov's mother who comes to visit him in St. Petersburg.
8. Luzhin - Dounia's fiance who is selfish and manipulative.
9. Katerina Ivanovna Marmeladova - Sonya's stepmother who is consumed by her own pride and vanity.
10. Nastasya Petrovna - A troubled woman who works as a servant for Svidrigailov.
11. Ilya Petrovich - A corrupt and abusive police officer.
12. Zossimov - A doctor who treats Raskolnikov after his illness.
13. Nikolai Dementiev - A painter who is wrongly accused of the murders.
14. Pyotr Petrovich Luzhin - Luzhin's father who is a high-ranking government official.
15. Vasiliev - A clerk who works for Svidrigailov.
16. Andrey Semyonovich Lebezyatnikov - A boarder who lives in the same building as Raskolnikov and is interested in socialist theories.
17. Lizaveta Ivanovna - The pawnbroker's sister who is also murdered by Raskolnikov.
18. Zametov - A young police officer who assists Porfiry Petrovich in the investigation.
19. Praskovya Pavlovna - The landlady of the building where Raskolnikov lives.
20. Zamyotov - Another police officer who is also involved in the investigation.

<a id="1.1"></a>
<h1 style="background-color:#33001a;font-family:cursive;font-size:200%;color:#fff;text-align:center;border-radius:40px;height:40px;line-height:40px;">1.1 Import libraries</h1>

In [ ]:
!pip install textacy
!pip install tqdm

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import regex as re
from wordcloud import WordCloud 
from collections import Counter 

from tqdm.auto import tqdm
from tqdm.notebook import tqdm
tqdm.pandas()

import html

import spacy
from spacy.tokens import Span
from spacy import Language
from spacy import displacy

from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.en.stop_words import STOP_WORDS as spacy_stopwords


# charachet normalization with textacy
import textacy
import textacy.preprocessing as tprep
import nltk


import requests
import io
import re
import random

In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
def clean(text):
    """
    A function that takes a string of text as input and applies several regular expression patterns to remove unwanted 
    characters or patterns from the text.

    Args:
    - text: A string of text to be cleaned.

    Returns:
    - A cleaned version of the input text with unwanted characters or patterns removed.
    """
  
    text = re.sub(r'<[^<>]*>', ' ', text)  # removes HTML tags
    
    text = re.sub(r'\[([^\[\]]*)\]\([^\(\)]*\)', r'\1', text)  # removes markdown links
    
    text = re.sub(r'\[[^\[\]]*\]', ' ', text)  # removes markdown reference links
   
    text = re.sub(r'("|_)', ' ', text)  # removes quotes and underscores
    
    text = re.sub(r'(?:^|\s)[&#<>{}\[\]+|\\:-]{1,}(?:\s|$)', ' ', text)  # removes special characters and symbols
    
    text = re.sub(r'(?:^|\s)[\-=\+]{2,}(?:\s|$)', ' ', text)  # removes consecutive equal signs or dashes
    
    text = re.sub(r'\s+', ' ', text)  # replaces multiple white spaces with a single space
   
    text = re.sub(r'\--+', ' ', text)  # replaces multiple dashes with a single space
    
    return text.strip()  # removes any leading or trailing white spaces from the cleaned text


def normalize(text):
    """
    A function that takes a string of text as input and applies several normalization techniques to standardize 
    the text for further processing or analysis.

    Args:
    - text: A string of text to be normalized.

    Returns:
    - A normalized version of the input text.
    """

    text = tprep.normalize.hyphenated_words(text)  # replaces hyphenated words with a single word
    text = tprep.normalize.quotation_marks(text)  # replaces different types of quotation marks with standard quotation marks
    text = tprep.normalize.unicode(text)  # replaces unicode characters with their ASCII equivalents
    text = tprep.remove.accents(text)  # removes diacritical marks from accented characters
    return text   


def count_words(df, column='tokens', preprocess=None, min_freq=2):
    """
    A function that takes a pandas DataFrame, a column name, a preprocess function, and a minimum frequency count as input, 
    and returns a sorted DataFrame of the most frequently occurring words in the specified column.

    Args:
    - df: A pandas DataFrame containing the text data.
    - column: A string representing the column name that contains the text data to be analyzed. Defaults to 'tokens'.
    - preprocess: A function that preprocesses the text data before counting the words. Defaults to None.
    - min_freq: An integer representing the minimum frequency count of a word to be included in the output. Defaults to 2.

    Returns:
    - A pandas DataFrame containing the most frequently occurring words in the specified column, sorted in descending order 
      of frequency.
    """
    
    # process tokens and update counter
    def update(doc):
        """
        A helper function that takes a document (a string) as input, preprocesses the document if a preprocess function is 
        provided, and updates the word counter with the resulting tokens.

        Args:
        - doc: A string representing a document to be processed.

        Returns:
        - None
        """
        tokens = doc if preprocess is None else preprocess(doc)
        counter.update(tokens)

    # create counter and run through all data
    counter = Counter()
    df[column].map(update)

    # transform counter into data frame
    freq_df = pd.DataFrame.from_dict(counter, orient='index', columns=['freq'])
    freq_df = freq_df.query('freq >= @min_freq')
    freq_df.index.name = 'token'

    return freq_df.sort_values('freq', ascending=False)


def wordcloud(word_freq, title=None, max_words=200, stopwords=None):
    """
    Create a word cloud visualization from a frequency counter.

    Parameters:
    ----------
    word_freq : dict or pd.Series
        A dictionary or pandas Series containing the word frequency data.
    title : str, optional
        The title of the word cloud plot. Default is None.
    max_words : int, optional
        The maximum number of words to be displayed in the word cloud plot. Default is 200.
    stopwords : set or None, optional
        A set of stop words to be filtered out from the word frequency data. Default is None.

    Returns:
    --------
    None, displays the plot

    Example: wordcloud(word_freq, 'Fruit Word Cloud', max_words=50, stopwords=stopwords)
    --------
    """

    # Create an instance of the WordCloud class with specific parameters.
    wc = WordCloud(width=800, height=400, 
                   background_color= "White", colormap="inferno", 
                   max_font_size=150, max_words=max_words)
    
    # Convert input word frequency data into a frequency counter.
    if type(word_freq) == pd.Series:
        counter = Counter(word_freq.fillna(0).to_dict())
    else:
        counter = word_freq

    # Filter out stop words from the frequency counter.
    if stopwords is not None:
        counter = {token:freq for (token, freq) in counter.items() 
                              if token not in stopwords}

    # Generate the word cloud visualization from the frequency counter.
    wc.generate_from_frequencies(counter)
 
    # Display the word cloud visualization with an optional title.
    plt.title(title) 
    plt.imshow(wc, interpolation='bilinear')
    plt.axis("off")
        

def extract_entities(text, include_types=None, sep=' '):

    """The function extracts named entities from a given text using spaCy and textacy libraries
    It returns a list of extracted entities after applying lemmatization and joining the 
    tokens with the specified separator
    Parameters:
    -----------
    text : str
    Input text from which named entities are to be extracted.
    include_types : list, optional
    List of entity types to include in the extraction. If not provided, all types will be included.
    sep : str, optional
    Separator to join the tokens of each named entity.

    Returns:
    --------
    entity_list : list
    A list of extracted named entities after applying lemmatization and joining the tokens with the specified separator.

    """

    # define the doc
    doc = nlp(text)

    # extract the entities
    ents = textacy.extract.entities(doc, 
    include_types=include_types, 
    exclude_types=None, 
    drop_determiners=True, 
    min_freq=1)

    # build the entity list
    entity_list = [sep.join([t.lemma_ for t in e]) for e in ents]
    return entity_list




def plot_bar_custom(df_toplot, desc,column,color1 = '#660066', color2 = '#33001a'):
    """
    Nice bar plot function that creates a horizontal bar plot 
    with dots for a given dataframe and column, 
    setting custom colors and style for the plot

    Parameters:
    -----------
    df : pandas DataFrame
    The DataFrame containing the data to plot
    desc : str
    The description or title for the plot
    column : str
    The column to use for plotting
    color1 : str, optional (default='#660066')
    The color to use for the bars and dots
    color2 : str, optional (default='#33001a')
    The color to use for the axes and text

    Returns:
    --------
    None, displays the plot

    """ 

    # set the style of the axes and the text color
    plt.rcParams['axes.edgecolor']=color2
    plt.rcParams['axes.linewidth']=0.8
    plt.rcParams['xtick.color']=color2
    plt.rcParams['ytick.color']=color2
    plt.rcParams['text.color']=color2


    # we first need a numeric placeholder for the y axis
    my_range=list(range(1,len(df_toplot.index)+1))

    fig, ax = plt.subplots(figsize=(10,5.5))

    ax.yaxis.grid(False, which='major')

    df_toplot = df_toplot.sort_values(by=column)
    # create for each expense type an horizontal line that starts at x = 0 with the length 
    # represented by the specific expense percentage value.
    plt.hlines(y=my_range, xmin=0, xmax=df_toplot[column], color=color1, alpha=0.5, linewidth=5)

    # create for each expense type a dot at the level of the expense percentage value
    plt.plot(df_toplot[column], my_range, "o", markersize=5, color=color1, alpha=1)

    # set labels
    ax.set_xlabel(column, fontsize=15, fontweight='black', color =color2)
    ax.set_ylabel('')

    # set axis
    ax.tick_params(axis='both', which='major', labelsize=15)
    plt.yticks(my_range, df_toplot.index)

    # add an horizonal label for the y axis 
    fig.text(-0.23, 0.96, desc, fontsize=15, fontweight='black', color = color2)

    # change the style of the axis spines
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    ax.spines['left'].set_bounds((1, len(my_range)))
    ax.set_xlim(0,80)

    ax.spines['left'].set_position(('outward', 8))
    ax.spines['bottom'].set_position(('outward', 5))


<a id="1.3"></a>
<h1 style="background-color:#33001a;font-family:cursive;font-size:200%;color:#fff;text-align:center;border-radius:40px;height:40px;line-height:40px;">SpacyLibrary</h1>

<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#600080;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">
        <p style="padding: 10px;
              color:white;"> SpaCy is an open-source natural language processing (NLP) library that provides a wide range of features for processing text. Its pipeline consists of several steps, each of which is responsible for a specific task.
</div>

In [ ]:
# check spacy version
assert spacy.__version__[0] >= '3'

In [ ]:
nlp = spacy.load("en_core_web_md")
# not in book: make sure stop words are available
nltk.download('stopwords')

stopwords = set(nltk.corpus.stopwords.words('english'))

<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#600080;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">
        <p style="padding: 10px;
              color:white;"> The en_core_web_md model in spaCy is an English language pipeline optimized for running on a CPU. It consists of the following components:
</div>

1. **Tokenizer** (tok2vec): The tokenizer component is responsible for breaking up the input text into individual tokens or words. In the en_core_web_md model, the tokenizer is implemented using a neural network architecture called tok2vec.

2. **Part-of-speech Tagger** (tagger): This component assigns each token a part-of-speech tag, such as noun, verb, or adjective. The tagger in en_core_web_md uses a convolutional neural network to predict the POS tags.

3. **Dependency Parser** (parser): The parser analyzes the grammatical structure of the sentence and identifies the relationships between the words. The parser in en_core_web_md is based on a neural network architecture called a transition-based parser.

4. **Sentence Boundary Detector** (senter): The sentence boundary detector identifies the boundaries between sentences in the input text. In en_core_web_md, this is implemented using a rule-based approach.

5. **Named Entity Recognizer** (ner): The NER component identifies and categorizes named entities in the input text, such as people, organizations, and locations. The NER in en_core_web_md uses a neural network architecture called a linear-chain conditional random field.

6. **Attribute Ruler** (attribute_ruler): This component allows for the manipulation of token attributes, such as adding new attributes or modifying existing ones. In en_core_web_md, the attribute ruler is used to add lemma and stop word attributes.

7. **Lemmatizer** (lemmatizer): The lemmatizer maps each token to its base form, or lemma. In en_core_web_md, the lemmatizer is implemented using a rule-based approach.

Overall, the en_core_web_md model provides a powerful suite of NLP tools that can be used for a variety of tasks, from simple tokenization to complex text analysis. It is optimized for running on a CPU, making it a good choice for applications where CPU resources are limited.

<a id="1.4"></a>
<h1 style="background-color:#33001a;font-family:cursive;font-size:200%;color:#fff;text-align:center;border-radius:40px;height:40px;line-height:40px;">Import the book</h1>

<table class="bibrec">
<colgroup>
<col class="narrow">
<col>
</colgroup>
<tr>
<th>Author</th>
<td>
<a href="https://www.gutenberg.org/ebooks/author/314" rel="marcrel:aut" about="/authors/314" typeof="pgterms:agent" itemprop="creator">Dostoyevsky, Fyodor, 1821-1881</a></td>
</tr><tr>
<th>Translator</th>
<td>
<a href="https://www.gutenberg.org/ebooks/author/2858" rel="marcrel:trl" about="/authors/2858" typeof="pgterms:agent" itemprop="creator">Garnett, Constance, 1861-1946</a></td>
</tr>
<tr>
<th>Uniform Title</th>
<td itemprop="alternativeHeadline">
Prestuplenie i nakazanie. English
</td>
</tr><tr>
<th>Title</th>
<td itemprop="headline">
Crime and Punishment
</td>
</tr>
<tr property="dcterms:language" datatype="dcterms:RFC4646" itemprop="inLanguage" content="en">
<th>Language</th>
<td>Inglese</td>
</tr>
<tr property="dcterms:subject" datatype="dcterms:LCC" content="PG">
<th>LoC Class</th>
<td>
<a href="/browse/loccs/pg">PG: Language and Literatures: Slavic (including Russian), Languages and Literature</a>
</td>
</tr>
<tr>
<th>Subject</th>
<td property="dcterms:subject" datatype="dcterms:LCSH">
<a class="block" href="https://www.gutenberg.org/ebooks/subject/1123">
Detective and mystery stories
</a>
</td>
</tr><tr>
<th>Subject</th>
<td property="dcterms:subject" datatype="dcterms:LCSH">
<a class="block" href="https://www.gutenberg.org/ebooks/subject/2430">
Psychological fiction
</a>
</td>
</tr><tr>
<th>Subject</th>
<td property="dcterms:subject" datatype="dcterms:LCSH">
<a class="block" href="https://www.gutenberg.org/ebooks/subject/3523">
Saint Petersburg (Russia) -- Fiction
</a>
</td>
</tr><tr>
<th>Subject</th>
<td property="dcterms:subject" datatype="dcterms:LCSH">
<a class="block" href="https://www.gutenberg.org/ebooks/subject/3550">
Murder -- Fiction
</a>
</td>
</tr><tr>
<th>Subject</th>
<td property="dcterms:subject" datatype="dcterms:LCSH">
<a class="block" href="https://www.gutenberg.orghttps://www.gutenberg.org/ebooks/subject/34776">
Crime -- Psychological aspects -- Fiction
</a>
</td>
</tr>
<tr>
<th>Category</th>
<td property="dcterms:type" datatype="dcterms:DCMIType">Text</td>
</tr>
<tr>
<th>EBook-No.</th>
<td>2554</td>
</tr>
<tr property="dcterms:issued" datatype="xsd:date" content="2006-03-28T00:00:00+00:00">
<th>Release Date</th>
<td itemprop="datePublished">28 mar 2006</td>
</tr>
<tr>
<th>Copyright Status</th>
<td property="dcterms:rights">Public domain in the USA.</td>
</tr>
<tr>
<th>Downloads</th>
<td itemprop="interactionCount">14065 downloads in the last 30 days.</td>
</tr>
<tr itemprop="offers" itemscope="itemscope" itemtype="http://schema.org/Offer">
<td colspan="2"><span itemprop="priceCurrency" content="USD"></span><span itemprop="price" content="$0.00"><em>Project Gutenberg books are always free!</em></span><span itemprop="availability" content="In Stock"><a href="http://schema.org/InStock"></a></span></td>
</tr>
</table>

<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#600080;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">
        <p style="padding: 10px;
              color:white;"> Download the book From Gutemberg
</div>

In [ ]:
# download the book from Gutenberg and store it in a text variable. 

url = "https://www.gutenberg.org/files/2554/2554-0.txt"
response = requests.get(url)
book = str(response.content,encoding='utf8' )

In [ ]:
print(book[:3000])

In [ ]:
# remove the preface
substring = "became great.”"
position = book.find(substring)
position = position + len(substring) - 1
book = book[position:]
print(book[:1000])

<a id="2"></a>
<h1 style="background-color:#33001a;font-family:cursive;font-size:200%;color:#fff;text-align:center;border-radius:40px;height:40px;line-height:40px;">Characters Identification</h1>

<a id="2.1"></a>
<h1 style="background-color:#33001a;font-family:cursive;font-size:150%;color:#fff;text-align:center;border-radius:40px;height:40px;line-height:40px;">Clean the Text</h1>

In [ ]:
# save the book in a dataframe
df_book = pd.DataFrame(columns=['text','clean_text'])
df_book.loc[0, 'text'] = book

In [ ]:
df_book['clean_text'] = df_book['text'].progress_map(clean)
df_book['clean_text'] = df_book['clean_text'].progress_map(normalize)
df_book.rename(columns={'text': 'raw_text', 'clean_text': 'text'}, inplace=True)
df_book.drop(columns = ['raw_text'], inplace = True)
print(df_book['text'])

<a id="2.2"></a>
<h1 style="background-color:#33001a;font-family:cursive;font-size:150%;color:#fff;text-align:center;border-radius:40px;height:40px;line-height:40px;">Named Entity Recognition</h1>

**Named Entity Recognition (NER)** is a subfield of Natural Language Processing (NLP) that focuses on identifying and categorizing named entities in text. Named entities are typically defined as words or phrases that refer to specific individuals, organizations, locations, products, or other types of entities that have a proper name.

NER is an important technique in many NLP applications, such as information retrieval, information extraction, **sentiment analysis**, and text classification. By identifying and categorizing named entities in text, NER systems can help to improve the accuracy of these applications.

There are several approaches to NER, including **rule-based systems**, statistical models, and deep learning techniques. Rule-based systems rely on manually defined rules to identify and categorize named entities in text. Statistical models use machine learning algorithms to learn patterns in text that correspond to named entities. Deep learning techniques, such as neural networks, can automatically learn representations of text that are useful for identifying named entities.

One of the key challenges in NER is dealing with **ambiguous entities**. For example, the name "Amazon" could refer to the online retailer, the rainforest, or the river. NER systems must be able to disambiguate these entities based on the context in which they appear.


In [ ]:
text = "Raskolnikov recognised Katerina Ivanovna at once. She was a rather tall, \
slim and graceful woman, terribly emaciated, with magnificent dark brown \
hair and with a hectic flush in her cheeks."
doc = nlp(text)
displacy.render(doc, style='ent', jupyter=True)

<a id="2.3"></a>
<h1 style="background-color:#33001a;font-family:cursive;font-size:150%;color:#fff;text-align:center;border-radius:40px;height:40px;line-height:40px;">Characters Identification</h1>

In [ ]:
df = pd.DataFrame(columns=['tokens','num_tokens'])

# This sets the maximum length of the spaCy language model to 1155653 characters. 
# to avoid memory errors when processing large amounts of text.
nlp.max_length = 1155653 

# Calculates the frequency of occurrence of each word in the 
# "tokens" column of the DataFrame and returns a new DataFrame with two "frequency"

df['tokens'] = df_book['text'].apply(extract_entities,args=(['PERSON']))
df['num_tokens'] = df['tokens'].map(len)
freq_df = count_words(df)
freq_df.head(15)

In [ ]:
# WordCloud
plt.figure(figsize=(16,8)) ###
freq_df = freq_df.apply(np.sqrt)
wordcloud(freq_df['freq'], title='Characters',max_words=50)

<a id="2.4"></a>
<h1 style="background-color:#33001a;font-family:cursive;font-size:150%;color:#fff;text-align:center;border-radius:40px;height:40px;line-height:40px;">Improve Characters Identification</h1>

<p style="font-size:130%; background-color:#f28482;border-radius: 0px 30px;text-align:center;font-family:Sans-serif;font-weight:bold">Normalizing</p>

**Article and  and  an
apostrophe-s**
​
The entity detection process can sometimes detect tokens that are not part of the actual entity name, such as in the case of "Katerina Ivanovna's". 
​
The same issue occurs with tokens like "the" in "the Italian Stock Exchange", which may or may not be considered part of the entity name. To simplify the linking of entity mentions, it is often helpful to remove these types of tokens, including articles and apostrophe-s, from the entity names. 
​
However, it's important to note that such rules may not always be accurate, as seen with entities like "The avengers" or "McDonald's", where preserving the article or apostrophe-s is necessary. In these cases, exceptions to the rules must be defined to avoid errors in entity linking.

In [ ]:
test_string = 'Katerina Ivanovna \'s'
doc = nlp(test_string)
displacy.render(doc, style='ent', jupyter=True)
print(*[([t.text for t in e], e.label_) for e in doc.ents], sep='\n')

In [ ]:
@Language.component("norm_entities")
def norm_entities(doc):
    """
    A custom spaCy component to normalize entities in a document by removing leading
    articles and trailing particles like 's.

    Args:
        doc (spacy.tokens.Doc): The input spaCy Doc object.

    Returns:
        spacy.tokens.Doc: The modified spaCy Doc object with normalized entities.
    """
    # function code here

    ents = []
    for ent in doc.ents:
        if ent[0].pos_ == "DET": # leading article
            ent = Span(doc, ent.start+1, ent.end, label=ent.label)
        if len(ent) > 0:
            if ent[-1].pos_ == "PART": # trailing particle like 's
                ent = Span(doc, ent.start, ent.end-1, label=ent.label)
            ents.append(ent)
    doc.ents = tuple(ents)
    return doc 

In [ ]:
# add the pipeline norm_entities

if nlp.has_pipe('norm_entities'): 
    _ = nlp.remove_pipe('norm_entities') 
nlp.add_pipe('norm_entities')

In [ ]:
# Example of norm_entities
test_string = 'Katerina Ivanovna \'s'
doc = nlp(test_string)
displacy.render(doc, style='ent', jupyter=True)
print(*[([t.text for t in e], e.label_) for e in doc.ents], sep='\n')

<p style="font-size:130%; background-color:#f28482;border-radius: 0px 30px;text-align:center;font-family:Sans-serif;font-weight:bold">Merge Entities</p>

**Compound names** like "Katerina Ivanovna" like
the previous example is trated as single tokens because it simplifies the sentence structure. spaCy pro‐vides a built-in pipeline function  **merge_entities**  for that purpose. We add it to our
NLP pipeline and get exactly one token per named-entity.

In [ ]:
from spacy.pipeline import merge_entities

# add the pipeline merge_entities

if nlp.has_pipe('merge_entities'): 
    _ = nlp.remove_pipe('merge_entities') 
nlp.add_pipe('merge_entities')

doc = nlp(text)
print(*[(t.text, t.ent_type_) for t in doc if t.ent_type_ != ''])

<p style="font-size:130%; background-color:#f28482;border-radius: 0px 30px;text-align:center;font-family:Sans-serif;font-weight:bold">Repeat the entity recognition with new pipelines step</p>

In [ ]:
df = pd.DataFrame(columns=['tokens','num_tokens'])

# calculates the frequency of occurrence of each word in the "tokens" 
# column of the DataFrame and returns a new DataFrame with frequency

df['tokens'] = df_book ['text'].apply(extract_entities,args=(['PERSON']))
df['num_tokens'] = df['tokens'].map(len)
freq_df = count_words(df)
freq_df.head(5)

In [ ]:
plt.figure(figsize=(16,8)) ###
freq_df = freq_df.apply(np.sqrt)
wordcloud(freq_df['freq'], title='Characters',max_words=50)

<a id="2.5"></a>
<h1 style="background-color:#33001a;font-family:cursive;font-size:150%;color:#fff;text-align:center;border-radius:40px;height:40px;line-height:40px;">Name Variation Resolution</h1

**Name Variation Resolution** is the process of identifying and merging different variations or representations of the same entity name into a single, canonical form. This is a common problem in natural language processing, especially in tasks such as named entity recognition and entity linking, where it is important to **correctly identify and link all mentions of the same entity** across different documents or contexts. 

**Name variations can include different spellings, abbreviations, aliases, titles, honorifics**, etc. Effective name variation resolution involves both rule-based and machine learning approaches, and often involves the use of external knowledge sources such as dictionaries, gazetteers, and knowledge graphs.

**We will adpt a simple rule-based approch.**

In [ ]:
# Find all the name that have common words

# Tokenize the index values into sets of words
tokenized_index = [set(x.split()) for x in freq_df.index]

# Find index entries that have words in common
common_word_pairs = []
for i, index_i in enumerate(tokenized_index):
    for j, index_j in enumerate(tokenized_index[i+1:], start=i+1):
        common_words = index_i.intersection(index_j)
        if common_words:
            common_word_pairs.append((freq_df.index[i], freq_df.index[j], common_words))

# Print the pairs of index values and the common words between them
for pair in common_word_pairs:
    print(f"{pair[0]} and {pair[1]} have these words in common: {pair[2]}")

In [ ]:
# find the my_rules dictionary values in the dataframe freq_df index
# replace the dataframe freq_df index found with the corresponding my_rules dictionary key

freq_df = freq_df.copy()

# Define the dictionary rules

my_rules = {
    'Afanasy Ivanovitch Vahrushin': ['Afanasy Ivanovitch' ],
    'Avdotya Romanovna':['Dounia'],
    'Dmitri Prokofitch':['Dmitri'],
    'Pyotr Petrovitch Luzhin':['Pyotr Petrovitch','Luzhin'],
    'Rodion Romanovitch Raskolnikov':['Rodion Romanovitch', 'Raskolnikov'],
    'Sofya Semyonovna Marmeladov':['Sofya Semyonovna'],
    'Amalia Ludwigovna':['Ludwigovna'],
    'Afanasy Ivanovitch Vahrushin':['Vahrushin'],
    'Sofya Semyonovna Marmeladov':['Sofya Semyonovna','Semyonovna Marmeladov'],
    'Resslich':['Madame Resslich'],
    'Alyona Ivanovna': ['pawnbroker'],
    'Porfiry Petrovitch': ['Porfiry']
    
}

# Apply the rules to the index of freq_df
for key, values in my_rules.items():
    for value in values:
        if value in freq_df.index:
            freq_df.rename(index={value: key}, inplace=True)


In [ ]:
freq_df = freq_df.groupby(by=freq_df.index ).sum()

<a id="2.6"></a>
<h1 style="background-color:#33001a;font-family:cursive;font-size:150%;color:#fff;text-align:center;border-radius:40px;height:40px;line-height:40px;">Final Word Cloud</h1>

In [ ]:
# select the 15 most important characters
freq_df = freq_df.sort_values(by = 'freq', ascending= True).iloc[-15:]
freq_df = freq_df.sort_values(by = 'freq', ascending= False)
freq_df

In [ ]:
plt.figure(figsize=(16,8)) ###
freq_df = freq_df.apply(lambda x: x*2)
wordcloud(freq_df['freq'], title='Characters',max_words=20)

In [ ]:
plot_bar_custom(freq_df, 'Character Name','freq')

<a id="2.7"></a>
<h1 style="background-color:#33001a;font-family:cursive;font-size:150%;color:#fff;text-align:center;border-radius:40px;height:40px;line-height:40px;">Reference</h1>


[Spacy Library]('https://spacy.io/') 

[TextAcy]('https://pypi.org/project/textacy/')

[Wordcloud]('https://pypi.org/project/wordcloud/')

[Hands On Natural Language Processing (NLP) using Python]('https://www.udemy.com/course/hands-on-natural-language-processing-using-python/') 

[Blueprints  for Text Analytics Using Python]('https://www.oreilly.com/library/view/blueprints-for-text/9781492074076/') 

[Regex101]('https://regex101.com/r/rX1tE6/7') 

[Beautiful bar plots matplotlib]('https://scentellegher.github.io/visualization/2018/10/10/beautiful-bar-plots-matplotlib.html') 